In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import numpy as np
import torch
ora_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/wanda/90/8_self_attn.v_proj_ora_W.npy')
new_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/wanda/90/8_self_attn.v_proj_new_W.npy')
test_inps = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/wanda/90/8_self_attn.v_proj_test_inps.npy')
test_outs = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/wanda/90/8_self_attn.v_proj_test_outs.npy')

ora_W = torch.tensor(ora_W).to(torch.float16).cuda()
new_W = torch.tensor(new_W).to(torch.float16).cuda()
test_inps = torch.tensor(test_inps).to(torch.float16)
test_outs = torch.tensor(test_outs).to(torch.float16)

In [13]:
print((new_W == 0).sum()/ new_W.numel())
print((ora_W == 0).sum()/ ora_W.numel())

tensor(0.9000, device='cuda:0')
tensor(1.8311e-06, device='cuda:0')


In [14]:
def get_error(ora_W, new_W, test_inps, test_outs):
    results = 0
    for i in range(test_inps.shape[0]):
        inp = test_inps[i].unsqueeze(0).cuda()
        out = test_outs[i].unsqueeze(0).cuda()
        result = torch.norm((new_W @ inp) - (out)) / torch.norm(out)
        results += result.item()
    results /= test_inps.shape[0]
    return result

init_W = ora_W.clone()
init_W[new_W == 0] = 0


error = get_error(ora_W, init_W, test_inps, test_outs)
print(f"Error: {error:.6f}")

error = get_error(ora_W, new_W, test_inps, test_outs)
print(f"Error: {error:.6f}")

Error: 0.556152
Error: 0.556152


In [15]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start
    A3 = (-A2) @ (I - H_A) @ A_start


    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k < 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start
            A3 = (-A2) @ (I - H_A) @ A_start



        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2 + delta**3 * A3)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断

        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break

        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=200)
print('time', time.time() - start)

cuda:0
tensor(0.1000, device='cuda:0')


100%|██████████| 200/200 [00:04<00:00, 44.24it/s]

time 4.537440299987793


In [16]:
error = get_error(ora_W, W_update, test_inps, test_outs)
print(f"Final Error: {error:.6f}")

Final Error: 0.151367
